In [ ]:
import numpy as np
import os
import natsort
import time
from my_dpca_module import myDPCA
sys.path.append(f'{ROOTDIR}2021-22_Attention/NP 2023-12/myDPCA')
from myDPCA import dpca, dpca_explainedVariance, dpca_custom_stimONLY

### Collect dPCs (variances) - decision components ONLY

In [ ]:
data_dir = os.path.join(ROOTDIR, '')
data_files = natsort.natsorted([os.path.join(data_dir, f) for f in os.listdir(data_dir)])

Lumped_dpca_decision = [[None for _ in range(3)] for _ in range(len(data_files) // 2)]
Lumped_dpca_independent = [[None for _ in range(3)] for _ in range(len(data_files) // 2)]
Lumped_expVar_decision = np.zeros(len(data_files) // 2)

# Loop through each data file
for ss in range(len(data_files) // 2):
    print(f"Processing session {ss+1}/{len(data_files)//2}")

    data_success = np.load(data_files[ss])
    data_fail = np.load(data_files[len(data_files) // 2 + ss])

    reso = 1 / 0.1
    time_vec = np.arange(1, data_success.shape[1] + 1) / reso - 5

    # Initialize firing rates with NaNs
    num_neurons = len(data_success)
    max_trials = max(data_success.shape[0], data_fail.shape[0])
    firing_rates = np.full((num_neurons, 2, data_success.shape[1], max_trials), np.nan)

    # Fill firing rate data
    for j in range(num_neurons):
        firing_rates[j, 0, :, :data_fail.shape[0]] = data_fail[j].T
        firing_rates[j, 1, :, :data_success.shape[0]] = data_success[j].T
    firing_rates_average = np.nanmean(firing_rates, axis=3)

    # Marginalization and DPCA parameters
    combined_params = [{1, (1, 2)}, {2}]
    marg_names = ['Decision', 'Condition-independent']
    marg_colours = np.array([[187, 20, 25], [150, 150, 150]]) / 256
    time_events = [time_vec[-1]]

    start_time = time.time()
    W, V, whichMarg = dpca(firing_rates_average, num_comps=20, combinedParams=combined_params, lambd=1e-4)
    print(f"dPCA completed in {time.time() - start_time:.2f} seconds")

    # Explained variance
    explVar = dpca_explainedVariance(firing_rates_average, W, V, combinedParams=combined_params)

    componentsToPlot, Zfull, options, Z = dpca_custom_stimONLY(
        firing_rates_average, W, V, plot_function=dpca_plot_default,
        explainedVar=explVar, marginalizationNames=marg_names,
        marginalizationColours=marg_colours, whichMarg=whichMarg,
        time=time_vec, timeEvents=time_events, timeMarginalization=3,
        legendSubplot=16
    )

    # Collect the relevant components
    for c, cc in enumerate(componentsToPlot):
        this_var = options.explainedVar.componentVar[cc] if options.explainedVar else None
        data = Zfull.take(c, axis=0)

        if c < 3:
            Lumped_dpca_decision[ss][c] = np.squeeze(data[0, :, :])
        else:
            Lumped_dpca_independent[ss][c - 3] = np.squeeze(data[0, :, :])

        if c == 0:
            Lumped_expVar_decision[ss] = this_var